In [1]:
import tensorflow as tf

## Get the dataset

In [3]:
# load sample as tf dataset
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12,
        label_name='label',
        na_value="0",
        num_epochs=1,
        ignore_errors=True)
    return dataset

train_dataset = get_dataset(
    #"H:/MachineLearningPractice/SparrowRecSys/NewCode/JavaPart/src/main/resources/webroot/sampledata/trainingSamples.csv"
    r"D:\MachineLearningPractice\SparrowRecSys\NewCode\JavaPart\src\main\resources\webroot\sampledata\trainingSamples.csv"
)
test_dataset = get_dataset(
#     "H:/MachineLearningPractice/SparrowRecSys/NewCode/JavaPart/src/main/resources/webroot/sampledata/testSamples.csv"
    r"D:\MachineLearningPractice\SparrowRecSys\NewCode\JavaPart\src\main\resources\webroot\sampledata\testSamples.csv"
)

## Get the features processed. 

In [4]:
genre_vocab = ['Film-Noir', 'Action', 'Adventure', 'Horror', 'Romance', 'War', 'Comedy', 'Western', 'Documentary',
               'Sci-Fi', 'Drama', 'Thriller',
               'Crime', 'Fantasy', 'Animation', 'IMAX', 'Mystery', 'Children', 'Musical']

GENRE_FEATURES = {
    'userGenre1': genre_vocab,
    'userGenre2': genre_vocab,
    'userGenre3': genre_vocab,
    'userGenre4': genre_vocab,
    'userGenre5': genre_vocab,
    'movieGenre1': genre_vocab,
    'movieGenre2': genre_vocab,
    'movieGenre3': genre_vocab
}

In [5]:
# all categorical features
categorical_columns = []
for feature, vocab in GENRE_FEATURES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list( ## to one-hot
        key=feature, vocabulary_list=vocab)
    emb_col = tf.feature_column.embedding_column(cat_col, 10)
    categorical_columns.append(emb_col)
# movie id embedding feature
movie_col = tf.feature_column.categorical_column_with_identity(key='movieId', num_buckets=1001) ## to one-hot
movie_emb_col = tf.feature_column.embedding_column(movie_col, 10)
categorical_columns.append(movie_emb_col)

In [6]:
# user id embedding feature
user_col = tf.feature_column.categorical_column_with_identity(key='userId', num_buckets=30001)
user_emb_col = tf.feature_column.embedding_column(user_col, 10)
categorical_columns.append(user_emb_col)

In [7]:
# all numerical features
numerical_columns = [tf.feature_column.numeric_column('releaseYear'),
                     tf.feature_column.numeric_column('movieRatingCount'),
                     tf.feature_column.numeric_column('movieAvgRating'),
                     tf.feature_column.numeric_column('movieRatingStddev'),
                     tf.feature_column.numeric_column('userRatingCount'),
                     tf.feature_column.numeric_column('userAvgRating'),
                     tf.feature_column.numeric_column('userRatingStddev')]

## Build and train the model

In [10]:
# embedding + MLP model architecture
model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(numerical_columns + categorical_columns),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])
# compile the model, set loss function, optimizer and evaluation metrics
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', tf.keras.metrics.AUC(curve='ROC'), tf.keras.metrics.AUC(curve='PR')])

# train the model
model.fit(train_dataset, epochs=5)

Epoch 1/5
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
7403/7403 [==============================] - 72s 9ms/step - loss: 6.6919 - accuracy: 0.5520 - auc: 0.5560 - auc_1: 0.6066
Epoch 2/5
7403/7403 [==============================] - 68s 9ms/step - loss: 0.6977 - accuracy: 0.6427 - auc: 0.6825 - auc_1: 0.7142 4s - loss: - E
Epoch 3/5
7403/7403 [==============================] - 62s 8ms/step - loss: 0.5669 - accuracy: 0.7087 - auc: 0.7694 - auc_1: 0.7903
Epoch 4/5
7403/7403 [==============================] - 57s 8ms/step - loss: 0.5267 - accuracy: 0.7397 - auc: 0.8090 - auc_1: 0.8275
Epoch 5/5
7403/7403 [==============================] - 68s 9ms/step - loss: 0.5015 - accuracy: 0.7539 - auc: 0.8300 - auc_1: 0.8522


In [11]:
# evaluate the model
test_loss, test_accuracy, test_roc_auc, test_pr_auc = model.evaluate(test_dataset)
print('\n\nTest Loss {}, Test Accuracy {}, Test ROC AUC {}, Test PR AUC {}'.format(test_loss, test_accuracy,
                                                                                   test_roc_auc, test_pr_auc))

Consider rewriting this model with the Functional API.
1870/1870 [==============================] - 10s 3ms/step - loss: 0.5997 - accuracy: 0.6941 - auc: 0.7531 - auc_1: 0.7813


Test Loss 0.5997158885002136, Test Accuracy 0.6940730810165405, Test ROC AUC 0.7531378865242004, Test PR AUC 0.7812909483909607


In [12]:
# print some predict results
predictions = model.predict(test_dataset)

for prediction, goodRating in zip(predictions[:12], list(test_dataset)[0][1][:12]):
    print("Predicted good rating: {:.2%}".format(prediction[0]),
          " | Actual rating label: ",
          ("Good Rating" if bool(goodRating) else "Bad Rating"))

Consider rewriting this model with the Functional API.
Predicted good rating: 86.09%  | Actual rating label:  Good Rating
Predicted good rating: 97.90%  | Actual rating label:  Good Rating
Predicted good rating: 75.16%  | Actual rating label:  Good Rating
Predicted good rating: 91.10%  | Actual rating label:  Good Rating
Predicted good rating: 37.20%  | Actual rating label:  Bad Rating
Predicted good rating: 66.78%  | Actual rating label:  Bad Rating
Predicted good rating: 61.58%  | Actual rating label:  Good Rating
Predicted good rating: 64.67%  | Actual rating label:  Bad Rating
Predicted good rating: 52.10%  | Actual rating label:  Good Rating
Predicted good rating: 81.53%  | Actual rating label:  Good Rating
Predicted good rating: 92.83%  | Actual rating label:  Good Rating
Predicted good rating: 67.02%  | Actual rating label:  Good Rating


## Some other codes

You can test out some detail parts in previous code. 

AttributeError: 'PrefetchDataset' object has no attribute 'head'